In [ ]:
%load_ext autoreload
%autoreload 2

from loader import *
from models import *
from utils import *
from plotting import *

folder_path = 'E:\Documents\Anaconda\Jupyterkod\Exjobb\Egen kod\Exjobb\From erik\Transfer'
list_paths = os.listdir(folder_path)   
image = cv2.imread(folder_path +'\\' + list_paths[20], 0)
plt.figure(figsize=(20,20))

plt.imshow(image, cmap='gray')

In [ ]:
col_delete_list=[0, 140, 1150, 1280]
row_delete_list=[0, 200, 350, 720, 880, 1024]
plt.figure(figsize=(20,20))

img = crop_and_append_image(image=image, col_delete_list=col_delete_list, row_delete_list=row_delete_list, print_shape=True)
plt.imshow(img, cmap='gray')

In [ ]:
im_size_width, im_size_height, radius = 1008, 304, (1.5e-7, 9e-7, 1.5e-7)
plankton = stationary_ellipsoid_plankton(im_size_height, im_size_width, radius, label=0)

gradient_amp = 0.5
microscope = plankton_brightfield(im_size_height, im_size_width, gradient_amp)

no_of_planktons = 50
sample = create_sample(plankton, no_of_planktons)

noise_amp = 2
norm_min, norm_max= -0.2, 1.2
image = create_image(noise_amp, sample, microscope, norm_min, norm_max)

plot_image(image)

In [ ]:
label = get_target_image(image.resolve())
label_function = get_target_image
plot_label(label_function, image)

In [ ]:
batch_function = create_custom_batch_function(image, 
                                              outputs=[0], 
                                              function_img=[Normalize_image],
                                              function_diff=[Normalize_image])

In [ ]:
from deeptrack.generators import ContinuousGenerator
generator = ContinuousGenerator(
    image,
    get_target_image,
    batch_function,
    batch_size=8,
    min_data_size=128,
    max_data_size=512
    )

In [ ]:
no_of_inputs, number_of_outputs = 1, 2
model = generate_unet(im_size_height, im_size_width, no_of_inputs, number_of_outputs)
model = train_model(model, generator, patience=10, epochs=200, steps_per_epoch=10)

In [ ]:
load_path_model = 'E:\\Documents\\Anaconda\\Jupyterkod\\Exjobb\\Egen kod\\Exjobb\\Test simulations\\Transfer rep 5 and 6\\results\\ver1\\models\\seq_len1_b7.keras'
model = keras.models.load_model(load_path_model, custom_objects={'softmax_categorical':softmax_categorical})

In [ ]:
# outputs = ["img", "img", "img", "diff", "diff"]
# output_numbers = [0, 1, 2, [0,1], [1,2]]
folder_path = 'E:\\Documents\\Anaconda\\Jupyterkod\\Exjobb\\Egen kod\\Exjobb\\From erik\\Transfer'
# calculated_mean_image = get_mean_image(folder_path, im_size_width=1280, im_size_height=1024)
# remove_mean_func = lambda img, **kwargs: img - calculated_mean_image

im_stack = get_image_stack(
    outputs=[0], 
    folder_path=folder_path, 
    frame_im0=20, 
    im_size_width=1008, 
    im_size_height=304,
    im_resize_width=1280, 
    im_resize_height=1024,
    function_img=[Normalize_image, RemoveRunningMean, crop_and_append_image, lambda x: np.exp(x), Normalize_image],
    row_delete_list=row_delete_list, 
    col_delete_list=col_delete_list,
    path_folder=folder_path,
    tot_no_of_frames=40, 
    im_height=1024,
    im_width=1280
)
plt.figure(figsize=(20,20))
plot_image_stack(im_stack)
# plot_prediction(model=model, im_stack=im_stack)
# 0.01, 0.1, 0.3, 0.5, 0.7, 0.9
plt.figure(figsize=(20,20))
plt.imshow(model.predict(im_stack)[0,:,:,1]>0.5, cmap='gray')

In [ ]:
positions = extract_positions(
    no_of_frames=10, 
    outputs=[0], 
    folder_path=folder_path, 
    frame_im0=0, 
    im_size_width=1008, 
    im_size_height=304,
    im_resize_width=1280, 
    im_resize_height=1024,
    model=model, 
    layer=1, 
    value_threshold=0.9,
    prediction_size=0,
    function_img=[Normalize_image, RemoveRunningMean, crop_and_append_image, lambda x: np.exp(x), Normalize_image],
    row_delete_list=row_delete_list, 
    col_delete_list=col_delete_list,
    path_folder=folder_path,
    tot_no_of_frames=20,
    im_height=1024,
    im_width=1280)


In [ ]:
new_positions = fix_positions_from_cropping(positions, col_delete_list=col_delete_list, row_delete_list=row_delete_list)

In [ ]:
list_of_plankton = assign_positions_to_planktons(new_positions, max_dist=60, threshold = 10, extrapolate=True)
list_of_plankton = Interpolate_gaps_in_plankton_positions(list_of_plankton=list_of_plankton)
list_of_plankton = Trim_list_from_stationary_planktons(list_of_plankton=list_of_plankton, min_distance=5)

plankton_track, plankton_dont_track = split_plankton(percentage_threshold=0.0, list_of_plankton=list_of_plankton)

In [ ]:
plot_and_save_track(no_of_frames=10,
                    plankton_track=plankton_track,
                    plankton_dont_track=plankton_dont_track,
                    folder_path=folder_path,
                    frame_im0=0,
                    save_images=False,
                    show_plankton_track = True,
                    show_plankton_dont_track = 0,
                    show_numbers_track = 0,
                    show_numbers_dont_track = 0,
                    show_numbers_specific_plankton = False,
                    show_specific_plankton = False,
                    specific_plankton = None,
                    color_plankton_track = 'b',
                    color_plankton_dont_track = 'r',
                    color_specific_plankton = 'w',
                    im_size_width=1280, 
                    im_size_height=1024,
                    save_path = 'E:\\Documents\\Anaconda\\Jupyterkod\\Exjobb\\Egen kod\\Exjobb\\New imulations\\transfer\\track',
                    frame_name = 'track',
                    file_type = '.jpg')

In [ ]:
# Make_video(frame_im0=0,
#            folder_path='E:\\Documents\\Anaconda\\Jupyterkod\\Exjobb\\Egen kod\\Exjobb\\New imulations\\transfer\\track', 
#            save_path='E:\\Documents\\Anaconda\\Jupyterkod\\Exjobb\\Egen kod\\Exjobb\\New imulations\\transfer\\Test_video.avi', 
#            fps=7, 
#            no_of_frames=100)